In [1]:
def classify_query(text):
    text = text.lower()
    if "due date" in text or "premium" in text:
        return "due_date"
    elif "claim" in text:
        return "claim"
    elif "renew" in text:
        return "renewal"
    elif "cancel" in text or "cancellation" in text:
        return "cancellation"
    else:
        return "other"


In [2]:
from datetime import datetime

def extract_due_date(text):
    if "due date" in text.lower() or "premium" in text.lower():
        return datetime.strptime("2025-02-15", "%Y-%m-%d")
    return None


In [3]:
def analyze_sentiment(text):
    text = text.lower()
    if "urgent" in text or "immediate" in text:
        return "urgent"
    else:
        return "neutral"


In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores['neg'] > 0.5:
        return "urgent"
    else:
        return "neutral"


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [5]:
from datetime import datetime

def calculate_priority(call):
    now = datetime.now()
    priority = 0

    type_of_query = classify_query(call['text_input'])
    due_date = extract_due_date(call['text_input'])
    sentiment = analyze_sentiment(call['text_input'])

    if type_of_query == 'due_date':
        days_until_due = (due_date - now).days if due_date else float('inf')
        priority += 100 if days_until_due < 30 else 50
    elif type_of_query == 'claim':
        priority += 80
    elif type_of_query in ['renewal', 'cancellation']:
        priority += 60
    else:
        priority += 30

    if sentiment == 'urgent':
        priority += 50

    return priority

def schedule_calls(call_data):
    calls_with_priority = [(call, calculate_priority(call)) for call in call_data]
    return sorted(calls_with_priority, key=lambda x: x[1], reverse=True)

call_data = [
    {'text_input': "Hi, this is Rahul Sharma. Can you tell me the due date for my health insurance premium? My policy number is 87543210."},
    {'text_input': "I want to file a claim for my policy. It's urgent, please help me with the process."},
    {'text_input': "I need to renew my policy. What is the procedure?"},
]

sorted_calls = schedule_calls(call_data)
for call, priority in sorted_calls:
    print(f"Call: {call['text_input']}, Priority: {priority}")


Call: I want to file a claim for my policy. It's urgent, please help me with the process., Priority: 80
Call: I need to renew my policy. What is the procedure?, Priority: 60
Call: Hi, this is Rahul Sharma. Can you tell me the due date for my health insurance premium? My policy number is 87543210., Priority: 50
